<a href="https://colab.research.google.com/github/GeekyOkami/Football_Yolov8/blob/main/yolov8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.1/800.1 kB 6.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
## importing required libraries
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [3]:
## connecting to the google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

In [6]:
'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images


    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')



    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")



    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
train_test_split('/content/drive/MyDrive/Yolov8/data/') ### without negative images
# train_test_split('./data/','./negative_images/') ### if you want to feed negative images

------ PROCESS STARTED -------
--- This folder has a total number of 66 images---


  0%|          | 0/53 [00:00<?, ?it/s]

------ Training data created with 80% split 53 images -------


  0%|          | 0/13 [00:00<?, ?it/s]

------ Testing data created with a total of 13 images ----------
------ TASK COMPLETED -------


In [7]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.54 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.4/78.2 GB disk)


In [8]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/Yolov8/dataset.yaml epochs=10 imgsz=640 batch=8 project=/content/drive/MyDrive/Yolov8/training_results name=football


100% 21.5M/21.5M [00:00<00:00, 175MB/s]
Ultralytics YOLOv8.2.54 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/Yolov8/dataset.yaml, epochs=10, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/Yolov8/training_results, name=football, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fra

In [11]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/Yolov8/training_results/football/weights/best.pt conf=0.55 source=/content/drive/MyDrive/Yolov8/test_images


Ultralytics YOLOv8.2.54 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

image 1/9 /content/drive/MyDrive/Yolov8/test_images/Copy of 6ty_f_110_54r.jpg: 384x640 16 players, 66.7ms
image 2/9 /content/drive/MyDrive/Yolov8/test_images/Copy of fr_f_55.jpg: 384x640 11 players, 10.8ms
image 3/9 /content/drive/MyDrive/Yolov8/test_images/p1.jpg: 448x640 4 players, 64.5ms
image 4/9 /content/drive/MyDrive/Yolov8/test_images/p2.jpeg: 448x640 4 players, 1 football, 12.7ms
image 5/9 /content/drive/MyDrive/Yolov8/test_images/p3.jpg: 384x640 7 players, 11.5ms
image 6/9 /content/drive/MyDrive/Yolov8/test_images/p4.jpg: 384x640 (no detections), 10.7ms
image 7/9 /content/drive/MyDrive/Yolov8/test_images/p5.jpeg: 512x640 2 footballs, 65.5ms
image 8/9 /content/drive/MyDrive/Yolov8/test_images/p6.jpg: 576x640 5 players, 2 footballs, 67.7ms
image 9/9 /content/drive/MyDrive/Yolov8/test_images/p7.jpg: 320x640 7 play

In [13]:
!cp -r /content/runs/detect/predict2 /content/drive/MyDrive/Yolov8/output

In [14]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/Yolov8/training_results/football/weights/best.pt conf=0.77 source=/content/drive/MyDrive/Yolov8/vid


Ultralytics YOLOv8.2.54 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

video 1/1 (frame 1/139) /content/drive/MyDrive/Yolov8/vid/v1.mp4: 384x640 16 players, 93.4ms
video 1/1 (frame 2/139) /content/drive/MyDrive/Yolov8/vid/v1.mp4: 384x640 14 players, 1 football, 10.8ms
video 1/1 (frame 3/139) /content/drive/MyDrive/Yolov8/vid/v1.mp4: 384x640 14 players, 10.8ms
video 1/1 (frame 4/139) /content/drive/MyDrive/Yolov8/vid/v1.mp4: 384x640 16 players, 1 football, 10.8ms
video 1/1 (frame 5/139) /content/drive/MyDrive/Yolov8/vid/v1.mp4: 384x640 2 players, 10.8ms
video 1/1 (frame 6/139) /content/drive/MyDrive/Yolov8/vid/v1.mp4: 384x640 3 players, 10.8ms
video 1/1 (frame 7/139) /content/drive/MyDrive/Yolov8/vid/v1.mp4: 384x640 2 players, 13.2ms
video 1/1 (frame 8/139) /content/drive/MyDrive/Yolov8/vid/v1.mp4: 384x640 2 players, 11.1ms
video 1/1 (frame 9/139) /content/drive/MyDrive/Yolov8/vid/v1.mp4: 3

In [15]:
!cp -r /content/runs/detect/predict3 /content/drive/MyDrive/Yolov8/output